#### Install these packages 
```python
pip install pandas
pip install numpy
pip install openpyxl  

```

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path


In [2]:
data = pd.read_json('testdata.json')

In [3]:
data.keys()

Index(['id', 'payment_status', 'stall_order', 'user_info', 'order_id',
       'order_placed_timestamp'],
      dtype='object')

In [4]:
index = 0
local_var=locals()
clean_data = data.drop(columns=['id', 'order_placed_timestamp', 'user_info',
       'payment_status', 'order_id'])["stall_order"]

In [5]:
for i in range (1,13):
    var_str  = 'stall_'+'{0}'.format(i).zfill(2)
    local_var.__setitem__(var_str,[])


In [6]:
def item_list_sep (stall_data):
    item_list = []
    for key in stall_data['items_ordered']:
        name = stall_data['items_ordered'][key]["name"]
        qty = stall_data['items_ordered'][key]["qty"]
        item_list.append(f"{name}({qty})")
    return item_list

In [7]:
def item_total (stall_data):
    total = 0
    for key in stall_data['items_ordered']:
        value = int( stall_data['items_ordered'][key]["qty"]) * int(stall_data['items_ordered'][key]["price"])
        total = total + value
    return total


In [8]:
def stall_details (stallList,stall_data,index,id) :
    formatted_time = datetime.fromtimestamp(data['order_placed_timestamp'][index]["seconds"]).ctime()
    item_list = item_list_sep(stall_data[id])
    stall_total= item_total(stall_data[id])
    user_name = data['user_info'][index]['name']
    user_email = data['user_info'][index]['email']
    user_phoneNo = data['user_info'][index]['phoneNumber']
    stallList.append((str(data['order_id'][index]),formatted_time,stall_data[id]["status"],','.join(map(str,item_list)),user_name,user_email,user_phoneNo,stall_total))


In [9]:
for stall_name in clean_data:
   for id in stall_name:
      for val in range(1,13):
          val_str='stall-'+'{0}'.format(val).zfill(2)
          val_list = 'stall_'+'{0}'.format(val).zfill(2)
          if(id == val_str):
             stall_details(local_var[val_list],stall_name,index,id)
         
   index=index+1

In [10]:
for index in range(1,13):
    val_str='stall_'+'{0}'.format(index).zfill(2)
    export_file = str(Path.cwd())+'\\export-excel\\export-stall'+'{0}'.format(index).zfill(2)+'.xlsx'
    
    # checks whether the stall array is empty or not
    if bool(local_var[val_str]):
        sheet1_data = pd.DataFrame(local_var[val_str],columns=['order_id',"order placed time",'status','items ordered','name','email','phone number','total'])
        sheet2_data = sheet1_data[sheet1_data['status'] == 'served']
        sheet3_data = sheet1_data[sheet1_data['status'] == 'refunded']
        sheet4_data = sheet1_data[sheet1_data['status'] == 'cancelled']
        sheet5_data = sheet1_data[sheet1_data['status'] == 'inprogress']
        sheet6_data = pd.DataFrame({"status":sheet1_data['status'],"total":sheet1_data["total"]}).groupby(["status"]).sum()
        
        with pd.ExcelWriter(export_file) as writer:
            
            sheet1_data.to_excel(writer,sheet_name="All Orders",index=False)
            sheet2_data.to_excel(writer,sheet_name="Served Orders",index=False)
            sheet3_data.to_excel(writer,sheet_name="Refunded Orders",index=False)
            sheet4_data.to_excel(writer,sheet_name="Cancelled Orders",index=False)
            sheet5_data.to_excel(writer,sheet_name="Inprogress Orders",index=False)
            sheet6_data.to_excel(writer,sheet_name="Total Amount")


In [11]:
val = 3

In [12]:
val > 5

False